# Deploying a Question Answering Model in Jarvis

[Transfer Learning Toolkit](https://developer.nvidia.com/transfer-learning-toolkit) (TLT) provides the capability to export your model in a format that can deployed using [Nvidia Jarvis](https://developer.nvidia.com/nvidia-jarvis), a highly performant application framework for multi-modal conversational AI services using GPUs. 

This tutorial explores taking an .ejrvs model, the result of `tlt question_answering export` command, and leveraging the Jarvis ServiceMaker framework to aggregate all the necessary artifacts for Jarvis deployment to a target environment. Once the model is deployed in Jarvis, you can issue inference requests to the server. We will demonstrate how quick and straightforward this whole process is. 

---
## Learning Objectives
In this notebook, you will learn how to:  
- Use Jarvis ServiceMaker to take a TLT exported .ejrvs and generate a model [resources](https://ngc.nvidia.com/resources/ea-jarvis-stage:jarvis_quickstart/)
- Deploy the model(s) locally  on the Jarvis Server
- Send inference requests from a demo client using Jarvis API bindings.

---
## Pre-requisites

To follow along, please make sure:
- You have access to NVIDIA NGC, and are able to download the Jarvis Quickstart [resources](https://ngc.nvidia.com/resources/ea-jarvis-stage:jarvis_quickstart/)
- Have an .ejrvs model file that you wish to deploy. You can obtain this from `tlt <task> export` (with `export_format=JARVIS`). Please refer the tutorial on *Question Answering using Transfer Learning Toolkit* for more details on training and exporting an .ejrvs model.

---
## Jarvis ServiceMaker
Servicemaker is the set of tools that aggregates all the necessary artifacts (models, files, configurations, and user settings) for Jarvis deployment to a target environment. It has two main components as shown below:

### 1. Jarvis-build

This step helps build a Jarvis-ready version of the model. It’s only output is an intermediate format (called a JMIR) of an end to end pipeline for the supported services within Jarvis. We are taking a QA model in consideration<br>

`jarvis-build` is responsible for the combination of one or more exported models (.ejrvs files) into a single file containing an intermediate format called Jarvis Model Intermediate Representation (.jmir). This file contains a deployment-agnostic specification of the whole end-to-end pipeline along with all the assets required for the final deployment and inference. Please check out the [documentation](http://docs.jarvis-ai.nvidia.com/release-1-0/service-asr.html#pipeline-configuration) to find out more.

In [ ]:
# IMPORTANT: Set the following variables

# Name of ServiceMaker Docker Image
JARVIS_SM_CONTAINER = "<add container name>"

# Directory where the .ejrvs model is stored $MODEL_LOC/*.ejrvs
MODEL_LOC = "<add path to model directory>"

# Name of the .erjvs file
MODEL_NAME = "<add model name>"

# Key that model is encrypted with
KEY = "<add encryption key used for trained model>"

In [ ]:
# Get the ServiceMaker docker
!docker pull $JARVIS_SM_CONTAINER

In [ ]:
# Syntax: jarvis-build <task-name> output-dir-for-jmir/model.jmir:key dir-for-ejrvs/model.ejrvs:key
!docker run --rm --gpus 1 -v $MODEL_LOC:/data $JARVIS_SM_CONTAINER -- \
            jarvis-build qa /data/question-answering.jmir:$KEY /data/$MODEL_NAME:$KEY

`NOTE:` Above, qa-model.ejrvs is the qa model obtained from `tlt question_answering export`

### 2. Jarvis-deploy

The deployment tool takes as input one or more Jarvis Model Intermediate Representation (JMIR) files and a target model repository directory. It creates an ensemble configuration specifying the pipeline for the execution and finally writes all those assets to the output model repository directory.

In [ ]:
# Syntax: jarvis-deploy -f dir-for-jmir/model.jmir:key output-dir-for-repository
!docker run --rm --gpus 1 -v $MODEL_LOC:/data $JARVIS_SM_CONTAINER -- \
            jarvis-deploy -f /data/question-answering.jmir:$KEY /data/models

---
## Start Jarvis Server
Once the model repository is generated, we are ready to start the Jarvis server. From this step onwards you need to download the Jarvis QuickStart Resource from NGC. 
Set the path to the directory here:

In [ ]:
 ### Set the Jarvis QuickStart directory
JARVIS_DIR = <path_to_jarvis_quickstart>

Next, we modify config.sh to enable relevant Jarvis services (nlp for Question Answering model), provide the encryption key, and path to the model repository (`jarvis_model_loc`) generated in the previous step among other configurations. 

For instance, if above the model repository is generated at `$MODEL_LOC/models`, then you can specify `jarvis_model_loc` as the same directory as `MODEL_LOC`

Pretrained versions of models specified in models_asr/nlp/tts are fetched from NGC. Since we are using our custom model, we can comment it in models_nlp (and any others that are not relevant to your use case). <br>

`NOTE:` You can perform this step of editing config.sh from outside this notebook.

#### config.sh snipet
```
# Enable or Disable Jarvis Services 
service_enabled_asr=false                                                      ## MAKE CHANGES HERE
service_enabled_nlp=true                                                      ## MAKE CHANGES HERE
service_enabled_tts=false                                                     ## MAKE CHANGES HERE

# Specify one or more GPUs to use
# specifying more than one GPU is currently an experimental feature, and may result in undefined behaviours.
gpus_to_use="device=0"

# Specify the encryption key to use to deploy models
MODEL_DEPLOY_KEY="tlt_encode"                                                  ## MAKE CHANGES HERE

# Locations to use for storing models artifacts
#
# If an absolute path is specified, the data will be written to that location
# Otherwise, a docker volume will be used (default).
#
# jarvis_init.sh will create a `jmir` and `models` directory in the volume or
# path specified. 
#
# JMIR ($jarvis_model_loc/jmir)
# Jarvis uses an intermediate representation (JMIR) for models
# that are ready to deploy but not yet fully optimized for deployment. Pretrained
# versions can be obtained from NGC (by specifying NGC models below) and will be
# downloaded to $jarvis_model_loc/jmir by `jarvis_init.sh`
# 
# Custom models produced by NeMo or TLT and prepared using jarvis-build
# may also be copied manually to this location $(jarvis_model_loc/jmir).
#
# Models ($jarvis_model_loc/models)
# During the jarvis_init process, the JMIR files in $jarvis_model_loc/jmir
# are inspected and optimized for deployment. The optimized versions are
# stored in $jarvis_model_loc/models. The jarvis server exclusively uses these
# optimized versions.
jarvis_model_loc="<add path>"                              ## MAKE CHANGES HERE (Replace with MODEL_LOC)                      
```

In [ ]:
# Ensure you have permission to execute these scripts
!cd $JARVIS_DIR && chmod +x ./jarvis_init.sh && chmod +x ./jarvis_start.sh

In [ ]:
# Run Jarvis Init. This will fetch the containers/models
# YOU CAN SKIP THIS STEP IF YOU DID JARVIS DEPLOY
!cd $JARVIS_DIR && ./jarvis_init.sh config.sh

In [ ]:
# Run Jarvis Start. This will deploy your model(s).
!cd $JARVIS_DIR && ./jarvis_start.sh config.sh

---
## Run Inference
Once the Jarvis server is up and running with your models, you can send inference requests querying the server. 

To send GRPC requests, you can install Jarvis Python API bindings for client. This is available as a pip .whl with the QuickStart.


In [ ]:
# IMPORTANT: Set the name of the whl file
JARVIS_API_WHL = "<add jarvis api .whl file name>"

In [ ]:
# Install client API bindings
!cd $JARVIS_DIR && pip install $JARVIS_API_WHL

The following code sample shows how you can perform inference using Jarvis Python API gRPC bindings:

In [ ]:
import grpc
import jarvis_api.jarvis_nlp_pb2 as jnlp
import jarvis_api.jarvis_nlp_pb2_grpc as jnlp_srv


grpc_server =  "localhost:50051"
channel = grpc.insecure_channel(grpc_server)
jarvis_nlp = jnlp_srv.JarvisNLPStub(channel)

req = jnlp.NaturalQueryRequest()

test_context = "In 2010 the Amazon rainforest experienced another severe drought, in some ways more extreme than the 2005 drought."\
                "The affected region was approximate 1,160,000 square miles (3,000,000 km2) of rainforest, compared to 734,000 square miles (1,900,000 km2)" \
                " in 2005. The 2010 drought had three epicenters where vegetation died off, whereas in 2005 the drought was focused on the southwestern part." \
                " The findings were published in the journal Science. In a typical year the Amazon absorbs 1.5 gigatons of carbon dioxide; during 2005" \
                "instead 5 gigatons were released and in 2010 8 gigatons were released."
req.query = "How many tons of carbon are absorbed the Amazon in a typical year?"
req.context = test_context
resp = jarvis_nlp.NaturalQuery(req)
print(resp)

`NOTE`: You could also run the above inference code from inside the Jarvis Client container. The QuickStart provides a script `jarvis_start_client.sh` to run the container. It has more examples for different services.

 You can stop all docker container before shutting down the jupyter kernel. **Caution: The following command will stop all running containers**

In [ ]:
!docker stop $(docker ps -a -q)

---
## What's next?
You could train your own custom models in TLT and deploy them in Jarvis! You could scale up your deployment using Kubernetes with the Jarvis AI Services Helm Chart, which will pull the relevant Images and download model artifacts from NGC, generate the model repository, start and expose the Jarvis speech services.